# 計算量算出

## OCSVM

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.font_manager
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_curve
from sklearn.svm import OneClassSVM
from skimage import measure
from sklearn import svm

import dataset
import mainmodel
import dataloader

In [3]:
Dataset = dataset.dataset("Obrid_AE", "data")
data= Dataset.read_train("sample_data")
# data =  torch.from_numpy((data).astype(np.float32)).clone()
# ae_data = data[:, np.newaxis, np.newaxis, :]#２次元のサイズでないとエラー

input_model = mainmodel.Autoencoder2()
input_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_model = mainmodel.Autoencoder2().to(device)
input_model.load_state_dict(torch.load("/Users/yukihorikawa/Desktop/LAB_LAST/AutoEncoder/program/syuron/model_data/20211206/newmodel.pth", map_location=device))

recon_list, encoded_list, input_list = dataloader.ocsvm_dataset(input_model, data)

X_train = encoded_list
clf = svm.OneClassSVM(nu=0.009, kernel="rbf", gamma=0.8)
clf.fit(X_train)
X_train = X_train[np.newaxis, :]#２次元のサイズでないとエラー

(500, 256)
1148 0
--------1/500--------
--------2/500--------
--------4/500--------
--------5/500--------
--------8/500--------
--------10/500--------
--------20/500--------
--------25/500--------
--------40/500--------
--------50/500--------
--------100/500--------
--------125/500--------
--------200/500--------
--------250/500--------
recon_list:(500, 256)
encoded_list:(500, 3)
input_list:(500,)


/Users/yukihorikawa/Desktop/LAB_LAST/AutoEncoder/AutoEncoder/dataloader.py:35: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  input_list = np.array(input_list)
/Users/yukihorikawa/Desktop/LAB_LAST/AutoEncoder/AutoEncoder/dataloader.py:35: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  input_list = np.array(input_list)


In [4]:
data.shape
num = 100
len = X_train.shape[0]
print(data.shape)
print(num)
print(len)

(500, 256)
100
1


In [5]:
%%time
for i in range(num):
    for j in range(len):
        y_pred_train = clf.predict(X_train[j])

CPU times: user 228 ms, sys: 2.34 ms, total: 231 ms
Wall time: 230 ms


## AE

In [6]:
Dataset = dataset.dataset("Obrid_AE", "data")
data= Dataset.read_train("sample_data")
data =  torch.from_numpy((data).astype(np.float32)).clone()
ae_data = data[:, np.newaxis, np.newaxis, :]#２次元のサイズでないとエラー
print(ae_data.shape)
print(num)
print(len)

(500, 256)
1148 0
torch.Size([500, 1, 1, 256])
100
1


In [7]:
%%time
num = 100
for i in range(num):
    for j in range(len):
        y_pred_train = input_model(ae_data[j]).detach().numpy()
        anomalypoint=sum(abs(y_pred_train.flatten()-ae_data[j].numpy()))

CPU times: user 40.7 ms, sys: 2.19 ms, total: 42.9 ms
Wall time: 42.8 ms
